# S3: AirBnb NYC 2024 Listings
by: Tam Trinh           
April 6, 2025         

<a id="0"></a> <br>
## Table of Contents  

- [Sprint Goals](#6)
- [Introduction](#1)
- [Set-up and Dataset Overview](#7)
- [Part 1: EDA](#2)
- [Part 2: Preprocessing](#3)   
- [Part 3: Modeling](#4) 
- [Stylized Visualizations](#8) 
- [Conclusion](#5)

<a id="7"></a>
## Set-up and Load Data

In [1]:
# imports for functionality and plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# optional imports
import warnings
import pickle

# imports for preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# imports for modeling
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from tempfile import mkdtemp
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# imports for model metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.inspection import permutation_importance

In [2]:
# ignore warnings
from warnings import filterwarnings
warnings.filterwarnings('ignore')

In [4]:
# load original dataset
airbnb_df = pd.read_csv("../data/new_york_listings_2024.csv")

# load dataset from previous notebook
aircopy = pd.read_csv("../data/eda_airbnb.csv")

<a id="3"></a>
## Part 2: Preprocessing

### Set up preprocessing workflow

To organize the workflow, the columns were sorted into the work that needed to be performed on them.

In [5]:
# make a copy of airbnb df to preprocess
prep_airbnb = aircopy.copy()

In [6]:
cols_2drop = ['id', 'name', 'host_id', 'host_name']

In [7]:
cols_2keep = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
              'reviews_per_month', 'calculated_host_listings_count',
              'availability_365', 'number_of_reviews_ltm', 'beds']

In [8]:
cols_2clean = ['bedrooms', 'baths']

In [9]:
cols_2edit = ['rating', 'neighbourhood', 'last_review']

In [10]:
cols_2onehot = ['neighbourhood_group', 'room_type', 'license']

_______________

### Columns to clean: bedrooms and baths

#### Bedrooms column
The bedrooms column was in string format, with some rows having "Studio" as a value. This column was cleaned by converting "Studio" to 0, which then made it possible to convert the column to integer type, and ready for modeling.

In [11]:
# change bedrooms "Studio" to 0
prep_airbnb.loc[(prep_airbnb["bedrooms"] == "Studio"), "bedrooms"] = 0

# convert col dtype to integer
prep_airbnb["bedrooms"] = prep_airbnb["bedrooms"].astype(int)

#### Baths column
The baths column was also in string format due to some values being "Not specified." To handle imputing an appropriate amount of baths, a dictionary was created using the average amount of baths per beds. The "Not specified" baths values were then filled in with the average baths expected for that amount of beds.

In [12]:
# create a dict to impute the avg bath per bed
baths_notnull = prep_airbnb[prep_airbnb["baths"] != "Not specified"] # subset the baths that are specified

baths_notnull["baths"] = baths_notnull["baths"].astype(float) # convert to float

baths_per_beds = baths_notnull.groupby("beds")["baths"].mean().round() # use the specified baths and beds to create a dict

bath_beds_dict = baths_per_beds.to_dict()


# for the missing baths, impute the avg bath based on the number of beds
prep_airbnb.loc[(prep_airbnb["baths"] == "Not specified"), "baths"] = prep_airbnb["beds"].map(bath_beds_dict)


# convert col dtype to integer
prep_airbnb["baths"] = prep_airbnb["baths"].astype(float)

__________________________

### Columns to edit: ratings, neighbourhood, and last_review

#### Rating column
The rating column contained values ranging from 1.75 to 5.0, as well as 3,593 values of "No rating" and 159 values of "New." Since there were a fair amount of non-numeric values, and they might be predictive, it was decided to keep them. To work around keeping non-numeric values with numeric values, the numeric ratings were binned. Most of the ratings fell between 4 and 5, so the bins were more granular between this range, and less granular with values less than 4. After binning, the whole column was transformed with One Hot Encoding. 

In [13]:
# clean ratings col

# create func to bin ratings
def bin_rating(rating):
    if rating == "No rating": # keep "No Rating" and "New " as strings
        return "No rating"
    elif rating == "New ":
        return "New"
    else:
        rating = float(rating)
        if rating == 5:
            return 5
        elif rating >= 4.8:
            return 4.8
        elif rating >= 4.6:
            return 4.6
        elif rating >= 4.4:
            return 4.4
        elif rating >= 4.2:
            return 4.2
        elif rating >= 4.0:
            return 4.0
        elif rating >= 3.0:
            return 3.0
        elif rating < 3.0:
            return "less3"
        else:
            return "Unknown rating"

In [14]:
# apply func to bin ratings
prep_airbnb["ratings_bin"] = prep_airbnb["rating"].apply(bin_rating)

# convert whole col to string
prep_airbnb["ratings_bin"] = prep_airbnb["ratings_bin"].astype(str)

In [15]:
# instantiate the OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

# fit the OneHotEncoder
ratings_encoded_array = one_hot_encoder.fit_transform(prep_airbnb[["ratings_bin"]])

# convert from sparse matrix to dense
ratings_dense_array = ratings_encoded_array.toarray()

# add prefix to feature names
feature_names = [f"ratings_{cat}" for cat in one_hot_encoder.categories_[0]]

# put into a dataframe
ratings_df = pd.DataFrame(ratings_dense_array, columns=feature_names, dtype=int)

In [16]:
ratings_df

,ratings_3.0,ratings_4.0,ratings_4.2,ratings_4.4,ratings_4.6,ratings_4.8,ratings_5,ratings_New,ratings_No rating,ratings_less3
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
20753,0,0,0,0,1,0,0,0,0,0
20754,0,0,0,1,0,0,0,0,0,0
20755,0,0,0,0,0,1,0,0,0,0
20756,0,0,0,0,0,0,1,0,0,0


#### Neighbourhood column
The neighbourhood column seemed to be important to the predication of price, however there were over 100 neighbourhoods, which were too many for dummy variables. So a new column based on the average price per neighbourhood was created to replace the neighbourhood name column.

In [17]:
# find the mean price per neighborhood
mean_prices_byneighborhood = prep_airbnb.groupby("neighbourhood")["price"].mean().round()

# create a dict based on mean price per neighborhood
prices_byneighborhood_dict = mean_prices_byneighborhood.to_dict()

# add column with the avg neighborhood price
prep_airbnb["neighborhood_avgprice"] = prep_airbnb["neighbourhood"].map(prices_byneighborhood_dict)

#### Last review column
The last review column initally contained the date of the last review, this was converted to two new columns, one containing the last review month, and the second containing the last review year.

In [18]:
# convert last_review from datatype object to datetime
prep_airbnb["last_review"] = pd.to_datetime(prep_airbnb["last_review"])

# last review month
prep_airbnb["last_review_month"] = prep_airbnb["last_review"].dt.month

# last review year
prep_airbnb["last_review_year"] = prep_airbnb["last_review"].dt.year

____________________

### Columns to One Hot Encode: neighbourhood group, room type, and license

#### Neighbourhood group column
The neighborhood group contains the name of the five NYC borough that the listing is located in. This column lended well to One Hot Encoding. 

In [19]:
aircopy["neighbourhood_group"].value_counts()

neighbourhood_group
Manhattan        8038
Brooklyn         7719
Queens           3761
Bronx             949
Staten Island     291
Name: count, dtype: int64

In [20]:
# instantiate the OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

# fit the OneHotEncoder neighborhood group
neigh_encoded_array = one_hot_encoder.fit_transform(prep_airbnb[["neighbourhood_group"]])

# convert from sparse matrix to dense
neigh_dense_array = neigh_encoded_array.toarray()

# add prefix to feature names
feature_names = [f"neighgroup_{cat}" for cat in one_hot_encoder.categories_[0]]

# put into a dataframe
neigh_group_df = pd.DataFrame(neigh_dense_array, columns=feature_names, dtype=int)

In [21]:
neigh_group_df

,neighgroup_Bronx,neighgroup_Brooklyn,neighgroup_Manhattan,neighgroup_Queens,neighgroup_Staten Island
0,0,1,0,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,1,0,0
4,0,0,1,0,0
...,...,...,...,...,...
20753,0,0,1,0,0
20754,0,0,1,0,0
20755,0,0,1,0,0
20756,0,0,1,0,0


#### Room type column
The room type contains information on whether the listing was an entire home, a private room, a shared room, or a hotel room. This column was converted into dummy variables with One Hot Encoding. 

In [22]:
# instantiate the OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

# fit the OneHotEncoder to room type
roomtype_encoded_array = one_hot_encoder.fit_transform(prep_airbnb[["room_type"]])

# convert from sparse matrix to dense
roomtype_dense_array = roomtype_encoded_array.toarray()

# add prefix to feature names
feature_names = [f"roomtype_{cat}" for cat in one_hot_encoder.categories_[0]]

# put into a dataframe 
roomtype_df = pd.DataFrame(roomtype_dense_array, columns=feature_names, dtype=int)


In [23]:
roomtype_df

,roomtype_Entire home/apt,roomtype_Hotel room,roomtype_Private room,roomtype_Shared room
0,0,0,1,0
1,1,0,0,0
2,1,0,0,0
3,0,0,1,0
4,1,0,0,0
...,...,...,...,...
20753,0,0,1,0
20754,1,0,0,0
20755,1,0,0,0
20756,1,0,0,0


#### License column
The license column indicated whether a listing had "No license," was license "Exempt," or listed the property's license number. Since the individual license number had little predictive power on the listing price, it was grouped into one group called "Has license." These groups were then One Hot Encoded. 

In [24]:
prep_airbnb["license"].value_counts()

license
No License            17569
Exempt                 2135
OSE-STRREG-0000068       14
OSE-STRREG-0008664        6
OSE-STRREG-0000437        6
                      ...  
OSE-STRREG-0000131        1
OSE-STRREG-0000288        1
OSE-STRREG-0000215        1
OSE-STRREG-0000985        1
OSE-STRREG-0000513        1
Name: count, Length: 879, dtype: int64

In [25]:
# create func to group license
def group_license(license):
    if license == "No License": 
        return "No License"
    elif license == "Exempt":
        return "Exempt"
    else:
        return "Has License"

In [26]:
# apply func to license
prep_airbnb["license"] = prep_airbnb["license"].apply(group_license)

In [27]:
# instantiate the OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

# fit the OneHotEncoder to license
license_encoded_array = one_hot_encoder.fit_transform(prep_airbnb[["license"]])

# convert from sparse matrix to dense
license_dense_array = license_encoded_array.toarray()

# add prefix to feature names
feature_names = [f"license_{cat}" for cat in one_hot_encoder.categories_[0]]

# put into a dataframe 
license_df = pd.DataFrame(license_dense_array, columns=feature_names, dtype=int)

In [28]:
license_df

,license_Exempt,license_Has License,license_No License
0,0,0,1
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1
...,...,...,...
20753,0,0,1
20754,0,0,1
20755,0,0,1
20756,0,0,1


--------

### Finalize cleaned_airbnb dataframe for modeling

A finalized cleaned_airbnb dataframe was assembled in preparation for modeling. The unneeded columns were dropped from the dataframe, and the OneHotEncoded dataframes containing the neighborhood group, room type, ratings, and license were concatenated. The finalized cleaned_airbnb dataframe contains 38 columns and all are either in integer or float format, ready for linear regression modeling.

In [29]:
prep_airbnb.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license', 'rating',
       'bedrooms', 'beds', 'baths', 'month', 'ratings_bin',
       'neighborhood_avgprice', 'last_review_month', 'last_review_year'],
      dtype='object')

In [30]:
# create list of additional cols to drop after feature eng.
cols_2drop_addt = ['neighbourhood_group', 'neighbourhood', 'room_type', 'last_review', 'license', 'rating', 'ratings_bin']

# add to the original cols to drop
cols_2drop.extend(cols_2drop_addt)

In [31]:
# create a copy called cleaned_airbnb to drop columns
cleaned_airbnb = prep_airbnb.copy()

In [32]:
# drop unneeded columns
cleaned_airbnb = cleaned_airbnb.drop(columns=cols_2drop)

In [33]:
# add in the OneHotEncoded dfs
cleaned_airbnb = pd.concat([cleaned_airbnb, neigh_group_df, roomtype_df, license_df, ratings_df], axis=1)

In [34]:
# check columns
cleaned_airbnb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   latitude                        20758 non-null  float64
 1   longitude                       20758 non-null  float64
 2   price                           20758 non-null  float64
 3   minimum_nights                  20758 non-null  int64  
 4   number_of_reviews               20758 non-null  int64  
 5   reviews_per_month               20758 non-null  float64
 6   calculated_host_listings_count  20758 non-null  int64  
 7   availability_365                20758 non-null  int64  
 8   number_of_reviews_ltm           20758 non-null  int64  
 9   bedrooms                        20758 non-null  int64  
 10  beds                            20758 non-null  int64  
 11  baths                           20758 non-null  float64
 12  month                           

In [35]:
# write to csv to be easily accessed later
cleaned_airbnb.to_csv("../data/cleaned_airbnb.csv", index=False)

__________________